In [12]:
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kilan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kilan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
df = pd.read_csv('amazon_reviews.csv')

In [4]:
df.head()

,overall,reviewText
0,4,No issues.
1,5,"Purchased this for my device, it worked as adv..."
2,4,it works as expected. I should have sprung for...
3,5,This think has worked out great.Had a diff. br...
4,5,"Bought it with Retail Packaging, arrived legit..."


In [13]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
stop_words = set(stopwords.words('english'))
def clean_data(df):
    df['reviewText'] = df['reviewText'].apply(lambda x: str(x).lower())
    df['reviewText'] = df['reviewText'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
    df['reviewText'] = df['reviewText'].apply(lambda x: word_tokenize(x))
    df['reviewText'] = df['reviewText'].apply(lambda x: [item for item in x if item not in stop_words])
    df['reviewText'] = df['reviewText'].apply(lambda x: ' '.join(x))
    return df

In [14]:
clean_data(df)

,overall,reviewText
0,4,issues
1,5,purchased device worked advertised never much ...
2,4,works expected sprung higher capacity think ma...
3,5,think worked greathad diff bran 64gb card went...
4,5,bought retail packaging arrived legit orange e...
...,...,...
4910,1,bought sandisk 16gb class 10 use htc inspire 3...
4911,5,used extending capabilities samsung galaxy not...
4912,5,great card fast reliable comes optional adapte...
4913,5,good amount space stuff want fits gopro say


In [20]:
from sklearn.model_selection import train_test_split
def train_test_val_split(X,y,test_size = 0.1, val_size = 0.1, random_state = 16):
    total_test_size = test_size + val_size
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=total_test_size, random_state=random_state)
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_size/total_test_size, random_state=random_state)
    return X_train,X_val,X_test,y_train,y_val,y_test

In [21]:
X_train, X_val, X_test, y_train, y_val, y_test = train_test_val_split(df['reviewText'],df['overall'])

In [22]:
X_train

1111    bought two 128 gb microsd cards formatted fat3...
4228    need get shots never enough memory cards reall...
493     gauge better faster class 4 6 use mobile saves...
4230                        used gopro camera works great
3117    much say except works great rw speed never see...
                              ...                        
4548               say open box plug works thats say item
635     great capacity purchased first purchased phone...
1345    bought 6yr olds nabi tablet thing works perfec...
581     bought new samsung galaxy 4 first one ordered ...
2169    bought save music mp3 movies mp4 asus memo pad...
Name: reviewText, Length: 3932, dtype: object

In [28]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as Op
torch.manual_seed(16)

In [ ]:
nn.LSTM()